In [1]:
import numpy as np
from scipy.ndimage import gaussian_filter1d





In [2]:
def double_rosin_thr(original_image, precision=0.01, gaussian_sigma=0.0):
    # Validate Arguments
    assert not (len(original_image.shape) != 2), "Error: Image must be grayscale."
    assert not (precision <= 0), "Error: Precision must be a positive value."
    assert not (gaussian_sigma < 0), "Error: Gaussian sigma must be a non-negative value."
    
    bins_number = round(1 / precision)
    
    # Preprocessing
    if np.max(original_image) > 1.001:
        factor = 255
        original_image = original_image / 255.0
    else:
        factor = 1

    # Histogram
    histogram = np.histogram(original_image.flatten(), bins=bins_number, range=(0, 1))[0]
    
    # Gaussian smoothing
    if gaussian_sigma > 0:
        histogram = gaussian_filter1d(histogram, gaussian_sigma)

    # Finding the peak
    hist_peak = np.argmax(histogram)

    # Finding the end of histogram
    hist_end = len(histogram) - 1
    while histogram[hist_end] == 0 and histogram[hist_end - 1] == 0 and hist_end > 1:
        hist_end -= 1
    if hist_end == 1:
        raise ValueError("Error: The histogram was empty or the image was 0-valued.")

    # Finding the high threshold
    high_thr = hist_peak + get_max_dist(histogram[hist_peak:hist_end]) - 1

    # Finding the low threshold
    low_thr = hist_peak + get_max_dist(histogram[hist_peak:high_thr]) - 1

    # Output formatting
    perc = np.array([low_thr, high_thr]) / bins_number
    double_thr = perc * factor

    return double_thr


In [3]:
def get_max_dist(func_values):
    x = 0
    y = len(func_values) - 1

    hx = func_values[x]
    hy = func_values[y]

    # Defining the line
    # (b) slope
    a = (hy - hx) / (y - x)
    # (a) abscises cutting point
    b = hx - (x * a)

    # Computing the best thresholds
    distances = np.zeros(len(func_values))
    max_distance = -1
    furthest_pos = 0
    for z in range(x, y + 1):
        # z is the candidate threshold
        hz = func_values[z]  # z's occurrence
        dist = abs(a * z - hz + b) / np.sqrt(a ** 2 + 1)
        distances[z] = dist
        if dist > max_distance:
            max_distance = dist
            furthest_pos = z

    return furthest_pos